In [102]:
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split

# Первичный анализ данных

In [2]:
data_path = '../data/full_data.csv'

In [3]:
df = pd.read_csv(data_path, index_col=0)

In [4]:
df.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


Оценим соотношение классов.

In [6]:
df.groupby('is_duplicate').count()

,name_1,name_2
is_duplicate,,
0,494161,494161
1,3658,3658


Выборка сильно несбалансирована.

Посмотрим, присутсвует ли что-то кроме латиницы.

In [107]:
u0 = map(lambda x: list(x), df['name_1'].tolist())
total = []
for lst in tqdm(u0, total=df['name_1'].shape[0]):
    total.extend(lst)

u1 = map(lambda x: list(x), df['name_2'].tolist())
for lst in tqdm(u1, total=df['name_2'].shape[0]):
    total.extend(lst)

  0%|          | 0/497819 [00:00<?, ?it/s]

  0%|          | 0/497819 [00:00<?, ?it/s]

In [108]:
set(total)

{' ',
 '"',
 '#',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 '@',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 '\\',
 ']',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '{',
 '\xa0',
 '«',
 '±',
 '»',
 '¿',
 'Ã',
 'Ç',
 'Í',
 'Ñ',
 'Ó',
 'Ö',
 'Ú',
 'Ü',
 'á',
 'ç',
 'è',
 'é',
 'í',
 'ñ',
 'ó',
 'õ',
 'ö',
 'ú',
 'ü',
 'ę',
 'İ',
 'Ł',
 'ł',
 'ő',
 'Ş',
 'ű',
 'Ё',
 'А',
 'Б',
 'В',
 'Г',
 'Д',
 'Е',
 'Ж',
 'З',
 'И',
 'Й',
 'К',
 'Л',
 'М',
 'Н',
 'О',
 'П',
 'Р',
 'С',
 'Т',
 'У',
 'Ф',
 'Х',
 'Ц',
 'Ч',
 'Ш',
 'Щ',
 'Ы',
 'Ь',
 'Э',
 'Ю',
 'Я',
 'а',
 'б',
 'в',
 'г',
 'д',
 'е',
 'з',
 'и',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',


Стоит убрать мусорные символы типа ", #, % и так далее. В эти символы также можно включить цифры и скобки:

In [121]:
trash_symbols = {
    '"', '#', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', ';', '<', '>', '?', '@', '[', '\\', ']', '`', '{', '\xa0', '«',
    '±', '»', '\u0e00', '‘', '’', '\u3000', '。', '﹠', '＆', '（', '）',
    '，'
}

Также в фразах присутсвует несколько языков:
* Латиница
* Кирилица
* Иероглифы
* Прочие

Разделим их и посмотрим на соотношение языков в датасете.

In [122]:
latin = {
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
    'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
    'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'
}

In [123]:
cyrillic = {
    'Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М',
    'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ы',
    'Ь', 'Э', 'Ю', 'Я', 'а', 'б', 'в', 'г', 'д', 'е', 'з', 'и', 'к', 'л',
    'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ш', 'ь', 'э'
}

In [124]:
unknown_1 = {
    '¿', 'Ã', 'Ç', 'Í', 'Ñ', 'Ó', 'Ö', 'Ú', 'Ü', 'á', 'ç', 'è', 'é', 'í',
    'ñ', 'ó', 'õ', 'ö', 'ú', 'ü', 'ę', 'İ', 'Ł', 'ł', 'ő', 'Ş', 'ű'
}

In [142]:
unknown_2 = {
    'ر', 'س', 'ف', 'ك', 'م', 'و', 'ي'
}

In [143]:
unknown_3 = {
    '上', '东', '京', '份', '会', '公', '北', '双', '司', '团', '式', '彤',
    '技', '料', '新', '方', '日', '有', '术', '材', '株', '水', '海', '社',
    '程', '股', '虹', '防', '限', '集', '雨'
}

In [144]:
(set(total)
 - latin
 - cyrillic
 - unknown_1
 - unknown_2
 - unknown_3
 - trash_symbols)

{' '}

Будем смотреть, какие языки охватывает каждая пара наименований.

In [156]:
def classify_language(
    string: str,
    languages_base: list,
    trash_symbols: set
) -> list:
    answer = []
    letters = set(string) - trash_symbols - {' '}
    for language in languages_base:
        answer.append(bool(language & letters) + 0)
    return answer

In [173]:
languages_base = [
    latin,
    cyrillic,
    unknown_1,
    unknown_2,
    unknown_3
]

languages_base_names = [
    'latin',
    'cyrillic',
    'unknown_1',
    'unknown_2',
    'unknown_3'
]

In [174]:
print(df['name_1'].iloc[0])
classify_language(df['name_1'].iloc[0], languages_base, trash_symbols)

Iko Industries Ltd.


[1, 0, 0, 0, 0]

In [205]:
df_copy = df.copy()

In [249]:
u = df_copy[['name_1', 'name_2']].apply(
    lambda row: classify_language(
        set(row['name_1']) | set(row['name_2']),
        languages_base,
        trash_symbols
    ) ,
    axis=1)

In [250]:
u.head(3)

pair_id
1    [1, 0, 0, 0, 0]
2    [1, 0, 0, 0, 0]
3    [1, 0, 0, 0, 0]
dtype: object

In [208]:
u2t = pd.DataFrame(u.tolist(), index=u.index, columns=languages_base_names)

In [209]:
u2t.head(3)

,latin,cyrillic,unknown_1,unknown_2,unknown_3
pair_id,,,,,
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0


In [210]:
df_copy[languages_base_names] = u2t[languages_base_names]

In [212]:
df_copy.head(3)

,name_1,name_2,is_duplicate,latin,cyrillic,unknown_1,unknown_2,unknown_3
pair_id,,,,,,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0,1,0,0,0,0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0,1,0,0,0,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0,1,0,0,0,0


Частота языков:

In [214]:
# В целом
df_copy.groupby('is_duplicate')[languages_base_names].sum()

,latin,cyrillic,unknown_1,unknown_2,unknown_3
is_duplicate,,,,,
0,494140,2305,5037,120,226
1,3621,149,41,1,2


In [222]:
# Пары с одним языком
one_language_df = df_copy[df_copy[languages_base_names].sum(axis=1) == 1]
one_language_df.groupby('is_duplicate')[languages_base_names].sum()

,latin,cyrillic,unknown_1,unknown_2,unknown_3
is_duplicate,,,,,
0,486646,21,0,0,0
1,3465,37,0,0,0


In [229]:
# Исходное число пар
df.groupby('is_duplicate').count()

,name_1,name_2
is_duplicate,,
0,494161,494161
1,3658,3658


В целом можно заключить, что взяв пары только с латинскими символами, мы не сильно потеряем от общей выборки. Учитывая то, насколько потенциально обременительно может быть тащить за собой все языки, это скорее правильное решение.

In [224]:
# Пары с различными языками
multi_language_df = df_copy[df_copy[languages_base_names].sum(axis=1) > 1]
print(multi_language_df.shape)
multi_language_df.groupby('is_duplicate')[languages_base_names].sum()

(7650, 8)


,latin,cyrillic,unknown_1,unknown_2,unknown_3
is_duplicate,,,,,
0,7494,2284,5037,120,226
1,156,112,41,1,2


In [231]:
df[['name_1', 'name_2']].duplicated().any()

False

# Очистка выборки

In [246]:
# Возьмем фразы только на латинском языке
target_columns = ['name_1', 'name_2', 'is_duplicate']
latin_df = one_language_df[one_language_df['latin'] == 1][target_columns]

In [277]:
def clear_string_from_symbols(string: str, symbols: set) -> str:
    res = ''.join(
        filter(
            lambda symb: symb not in symbols,
            list(string)
        )
    )
    return res

In [291]:
print(latin_df.iloc[25]['name_1'])
r = clear_string_from_symbols(latin_df.iloc[25]['name_1'], trash_symbols)
print(r)

S.R. International
SR International


In [279]:
latin_df_copy = latin_df.copy()

In [280]:
# Уберем мусорные символы
for col in ['name_1', 'name_2']:
    latin_df_copy[col] = latin_df_copy[col].apply(
        lambda val: clear_string_from_symbols(val, trash_symbols)
    )

In [282]:
latin_df.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0
2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0
3,"Rishichem Distributors Pvt., Ltd.",Dsa,0
4,Powermax Rubber Factory,Co. One,0
5,Tress A/S,Longyou Industries Park Zhejiang,0


In [281]:
latin_df_copy.head()

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd,Enormous Industrial Trade Pvt Ltd,0
2,Apcotex Industries Ltd,Technocraft Industries India Ltd,0
3,Rishichem Distributors Pvt Ltd,Dsa,0
4,Powermax Rubber Factory,Co One,0
5,Tress AS,Longyou Industries Park Zhejiang,0


In [283]:
latin_df_copy.shape

(490111, 3)

In [284]:
latin_df_copy.groupby('is_duplicate').count()

,name_1,name_2
is_duplicate,,
0,486646,486646
1,3465,3465


In [286]:
latin_df_copy.isna().any()

name_1          False
name_2          False
is_duplicate    False
dtype: bool

In [292]:
latin_df_copy.head(3)

,name_1,name_2,is_duplicate
pair_id,,,
1,Iko Industries Ltd,Enormous Industrial Trade Pvt Ltd,0
2,Apcotex Industries Ltd,Technocraft Industries India Ltd,0
3,Rishichem Distributors Pvt Ltd,Dsa,0


In [293]:
latin_df_copy.to_csv('clear_latin_data.csv', index=False)

# Разбиение данных на обучение и тест

Разделим выюорку на train и test сохранив процентное соотношение классов. Для теста выделим $10\%$ всей выборки.

In [294]:
ds_path = '../data/clear_latin_data.csv'
df_clear = pd.read_csv(ds_path)

In [295]:
df_clear.head(3)

,name_1,name_2,is_duplicate
0,Iko Industries Ltd,Enormous Industrial Trade Pvt Ltd,0
1,Apcotex Industries Ltd,Technocraft Industries India Ltd,0
2,Rishichem Distributors Pvt Ltd,Dsa,0


In [296]:
X, y = df_clear[['name_1', 'name_2']].values, df_clear[['is_duplicate']].values

In [297]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, shuffle=True, test_size=0.1, random_state=681)

In [298]:
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print()
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

X_train: (441099, 2)
X_test: (49012, 2)

y_train: (441099, 1)
y_test: (49012, 1)


In [299]:
y_train_coeff = round(
    y_train.sum() / (y_train.shape[0] - y_train.sum()), 4)
print(f'y_train: {y_train_coeff}')

y_test_coeff = round(
    y_test.sum() / (y_test.shape[0] - y_test.sum()), 4)
print(f'y_test: {y_test_coeff}')

y_train: 0.0071
y_test: 0.0071


Сохраним данные

In [300]:
df_train_data = pd.DataFrame(X_train, columns=['name_1', 'name_2'])
df_train_data['is_duplicate'] = y_train

df_test_data = pd.DataFrame(X_test, columns=['name_1', 'name_2'])
df_test_data['is_duplicate'] = y_test

In [301]:
df_train_data.to_csv('train_data.csv', index=False)

In [302]:
df_test_data.to_csv('test_data.csv', index=False)

# Проверка данных

In [307]:
train_data_path = '../data/train_data.csv'
test_data_path = '../data/test_data.csv'

In [308]:
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [309]:
print(train_data.shape)
train_s = train_data['is_duplicate'].sum()
round(train_s / (train_data['is_duplicate'].shape[0] - train_s), 4)

(441099, 3)


0.0071

In [310]:
print(test_data.shape)
test_s = test_data['is_duplicate'].sum()
round(test_s / (test_data['is_duplicate'].shape[0] - test_s), 4)

(49012, 3)


0.0071